# **ML-Based Intrusion Detection System (IDS)**
# Data Preparation and Exploratory Data Analysis (EDA)

### Gregorio Mendoza Serrano

First, let's import the necessary libraries and dependencies:

In [ ]:
# list of imports
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from sklearn.feature_selection import VarianceThreshold
import tensorflow as tf
from tensorflow.keras import layers, models
import plotly.express as px
import seaborn as sns


---

# Importing and Preparing the Dataset

---


Download the `IDS_Dataset.zip` file, which contains all the necessary .csv files for training and testing:

In [ ]:
!gdown 15bfx9AMWCCXLWDcX1LBraiwwP_YiNIv6

Downloading...
From (original): https://drive.google.com/uc?id=15bfx9AMWCCXLWDcX1LBraiwwP_YiNIv6
From (redirected): https://drive.google.com/uc?id=15bfx9AMWCCXLWDcX1LBraiwwP_YiNIv6&confirm=t&uuid=691ce30b-b903-4a85-a260-e150aac54353
To: /content/IDS_Dataset.zip
100% 241M/241M [00:03<00:00, 76.4MB/s]


Extract the downloaded .zip file in the directory of this Colab Notebook:

In [ ]:
import zipfile

# Unzip the archive
local_zip = './IDS_Dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

Perform a check of the names of the extracted files:

In [ ]:
import os

# Lista de archivos en el directorio base:
archivos = os.listdir('IDS_Dataset')

# Crear un DataFrame de Pandas con la lista de archivos y mostrarlo:
df_IDS_Files = pd.DataFrame({'Archivo': archivos})
df_IDS_Files

,Archivo
0,Friday-WorkingHours-Morning.pcap_ISCX.csv
1,Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
2,Thursday-WorkingHours-Afternoon-Infilteration....
3,Tuesday-WorkingHours.pcap_ISCX.csv
4,Thursday-WorkingHours-Morning-WebAttacks.pcap_...
5,Monday-WorkingHours.pcap_ISCX.csv
6,Wednesday-workingHours.pcap_ISCX.csv
7,Friday-WorkingHours-Afternoon-PortScan.pcap_IS...


Create a Pandas DataFrame that contains all the available data from the previous dataset:


In [ ]:
# Directorio donde se encuentran los archivos CSV
carpeta = 'IDS_Dataset'

df_IDS_combine = []
for archivo in df_IDS_Files['Archivo']:
    # Ubicación del archivo a leer:
    ruta_archivo = os.path.join(carpeta, archivo)
    # Leer el archivo CSV
    df_temp = pd.read_csv(ruta_archivo)
    # Agregar el DataFrame leído a la lista
    df_IDS_combine.append(df_temp)

# Combinar todos los DataFrames en uno solo
df_IDS_all = pd.concat(df_IDS_combine, ignore_index=True)

# Eliminar los DataFrames temporales
del df_IDS_combine
del df_temp

Display the resulting combined DataFrame:

In [ ]:
from sklearn.utils import shuffle

# Mezclar el DataFrame
df_IDS_all = shuffle(df_IDS_all, random_state=3)

df_IDS_all.head(10)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1090143,5355,99632,24,0,576,0,24,24,24.0,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
67513,53,186,2,2,104,224,52,52,52.0,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2113229,80,98463165,6,6,357,11595,339,0,59.5,136.958023,...,20,13007.0,0.0,13007,13007,98400000.0,0.0,98400000,98400000,DoS Hulk
2599722,22,627,2,1,8,2,6,2,4.0,2.828427,...,20,0.0,0.0,0,0,0.0,0.0,0,0,PortScan
1822668,53,82222,1,1,57,140,57,57,57.0,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
179581,443,3543632,5,1,135,46,46,6,27.0,20.124612,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
661395,53,243,2,2,68,158,34,34,34.0,0.000000,...,40,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
533657,992,49,2,0,4,0,2,2,2.0,0.000000,...,24,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2502610,53,50251,2,2,102,158,51,51,51.0,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
244343,80,3429510,5,0,30,0,6,6,6.0,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS


Create the dataset `df_ids`, which will be used for all subsequent cleaning and curating tasks. This dataset will contain all the records from the merged dataset, ensuring a balanced number of positive and negative (0/1) labels once the complete dataset is partitioned into various sub-datasets for model training.


In [ ]:
print('Shape of imported Pandas list =',df_IDS_all.shape)

# Eliminamos registros nulos y duplicados y creamos df_ids:
df_ids = df_IDS_all.drop_duplicates().dropna()
#df_ids = df_IDS_all.sample(frac=0.25, random_state=3)

# Presentamos por pantalla la cabecera:
print('Shape of processed Pandas list =',df_ids.shape)

Shape of imported Pandas list = (2830743, 79)
Shape of processed Pandas list = (2522009, 79)


In [ ]:
# Eliminamos el dataframe completo para liberar RAM:
del df_IDS_all

Display the description of each column in the dataset using `.describe()`:

In [ ]:
df_ids.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,...,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06
mean,8.701432e+03,1.658364e+07,1.027750e+01,1.156751e+01,6.116607e+02,1.813569e+04,2.311241e+02,1.919733e+01,6.347899e+01,7.728840e+01,...,6.006745e+00,-3.080742e+03,9.153450e+04,4.616959e+04,1.719345e+05,6.543215e+04,9.332884e+06,5.655225e+05,9.759082e+06,8.888401e+06
std,1.902225e+04,3.522618e+07,7.942294e+02,1.056668e+03,1.058573e+04,2.397602e+06,7.562104e+02,6.079830e+01,1.955137e+02,2.968147e+02,...,6.742531e+02,1.149482e+06,6.864883e+05,4.164856e+05,1.085317e+06,6.110134e+05,2.484306e+07,4.873014e+06,2.561220e+07,2.457630e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,2.080000e+02,2.000000e+00,1.000000e+00,1.200000e+01,6.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,5.058700e+04,2.000000e+00,2.000000e+00,6.600000e+01,1.560000e+02,4.000000e+01,2.000000e+00,3.613084e+01,0.000000e+00,...,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,5.330376e+06,6.000000e+00,5.000000e+00,3.320000e+02,9.910000e+02,2.020000e+02,3.700000e+01,5.200000e+01,7.417179e+01,...,3.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,1.200000e+08,2.197590e+05,2.919220e+05,1.290000e+07,6.554530e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.125597e+03,...,2.135570e+05,1.380000e+02,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08



---

# Data Cleaning and Curating

---

---

First, we display the names of all columns to determine how to improve their names:



In [ ]:
df_ids.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

We can see that many of them start with an empty field. We change the names of all columns that contain an empty character at the beginning and remove that space.

To make the names more manageable in subsequent code, we replace all spaces (" ") in the column names with underscores ("_").

In [ ]:
# Modificar los nombres de las columnas
for column in df_ids.columns:
    new_column_name = column.strip().replace(" ", "_")  # Eliminar espacios y cambiar espacios restantes por _
    df_ids.rename(columns={column: new_column_name}, inplace=True)  # Renombrar la columna

# Mostrar los nombres de las columnas después de la modificación
print("\nNombres de columnas después de la modificación:")
print(df_ids.columns)


Nombres de columnas después de la modificación:
Index(['Destination_Port', 'Flow_Duration', 'Total_Fwd_Packets',
       'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets',
       'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Max',
       'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean',
       'Fwd_Packet_Length_Std', 'Bwd_Packet_Length_Max',
       'Bwd_Packet_Length_Min', 'Bwd_Packet_Length_Mean',
       'Bwd_Packet_Length_Std', 'Flow_Bytes/s', 'Flow_Packets/s',
       'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min',
       'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max',
       'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std',
       'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags',
       'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length',
       'Bwd_Header_Length', 'Fwd_Packets/s', 'Bwd_Packets/s',
       'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean',
       'Packet_Length_Std', 'Packet_Length

As we can see, the formats of the names are more uniform and follow a logical pattern.

Before creating a correlation matrix, we remove fields with n/a values and assign values of 0 or 1 to the target category 'Label':


In [ ]:
# Eliminar filas con valores NaN
df_ids.dropna(inplace=True)

# Reemplazar los valores en la columna 'Label'
df_ids['Label'] = df_ids['Label'].replace({'BENIGN': 0})

# Convertir todos los demás valores a 1
df_ids['Label'] = df_ids['Label'].apply(lambda x: 1 if x != 0 else x)


We create an auxiliary dataset (*subset*) that contains only the first 500,000 entries to perform exploratory analysis more quickly:



In [ ]:
# Crear un nuevo DataFrame con los primeros 500000 registros
df_ids_subset = df_ids.head(500000)

We draw the correlation matrix using the `plotly` library, which allows us to see explicit values by moving the cursor over the cell and zoom in to observe the values more closely if necessary.


In [ ]:
df_ids_corr = df_ids_subset.corr()
fig = px.imshow(df_ids_corr, color_continuous_scale='Viridis', labels=dict(color='Correlación'))
fig.update_layout(title='Matriz de Correlación', width=1000, height=900)
fig.show()

We remove columns from the correlation matrix that contain NaN values (shown in gray) and regenerate the matrix:


In [ ]:
df_ids_filtered = df_ids_subset.copy()
df_ids_filtered.drop(columns=['Fwd_URG_Flags','CWE_Flag_Count','Bwd_Avg_Bytes/Bulk','Fwd_Avg_Bulk_Rate','Bwd_Avg_Packets/Bulk','Bwd_URG_Flags',
                     'Bwd_PSH_Flags','Bwd_Avg_Bulk_Rate','Fwd_Avg_Bytes/Bulk','Fwd_Avg_Packets/Bulk'], inplace=True)

We provide the correlation matrix with `plotly` after removing the previously mentioned features:


In [ ]:
df_ids_filtered_corr = df_ids_filtered.corr()
fig = px.imshow(df_ids_filtered_corr, color_continuous_scale='Viridis', labels=dict(color='Correlación'))
fig.update_layout(title='Matriz de Correlación', width=1000, height=900)
fig.show()

The next feature filter will be based on the definition of the features to rationally eliminate those that are coupled together by their technical definition.

The features removed, based on technical criteria and considering the cross-correlation values found in the previous correlation matrix, are as follows:

* Variables with minimum and maximum packet lengths (Min, Max), as these are represented in the corresponding mean and standard deviation features (Mean, Std).
* Variables that contain minimum and maximum times, for the same reason as the previous point.









In [ ]:
# Obtener una lista de las columnas que contienen 'Min' o 'Max' en su nombre
columnas_a_eliminar = [columna for columna in df_ids_filtered.columns if 'Min' in columna or 'Max' in columna]
df_ids_filtered.drop(columns=columnas_a_eliminar, inplace=True)

print('Características eliminadas:')
print(columnas_a_eliminar)
print(len(columnas_a_eliminar))

Características eliminadas:
['Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min', 'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Min_Packet_Length', 'Max_Packet_Length', 'Active_Max', 'Active_Min', 'Idle_Max', 'Idle_Min']
16


We redraw the correlation matrix:


In [ ]:
df_ids_filtered_corr = df_ids_filtered.corr()
fig = px.imshow(df_ids_filtered_corr, color_continuous_scale='Viridis', labels=dict(color='Correlación'))
fig.update_layout(title='Matriz de Correlación', width=1000, height=900)
fig.show()

print('Número de características:')
print(df_ids_filtered.shape[1])


Número de características:
53


We now perform a screening based solely on the correlation value of the dataset's features with the *target* 'Label'.
All features with a correlation of less than 0.2 with 'Label' (our probability of attack) will be discarded.


In [ ]:
# Obtener la columna de correlaciones entre 'Label' y todas las demás características
correlaciones_label = df_ids_filtered_corr.loc['Label']

# Filtrar las características con una correlación menor que 0.2 con 'Label':
caracteristicas_a_eliminar = correlaciones_label[abs(correlaciones_label) < 0.2].index.tolist()

# Eliminar las características seleccionadas del DataFrame
df_ids_filtered = df_ids_filtered.drop(columns=caracteristicas_a_eliminar)

# Mostrar las características eliminadas
print("Características eliminadas:")
print(caracteristicas_a_eliminar)
print("Número de características eliminadas:")
print(len(caracteristicas_a_eliminar))

Características eliminadas:
['Destination_Port', 'Total_Fwd_Packets', 'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std', 'Flow_Bytes/s', 'Flow_Packets/s', 'Fwd_IAT_Mean', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Fwd_PSH_Flags', 'Fwd_Header_Length', 'Bwd_Header_Length', 'Fwd_Packets/s', 'Bwd_Packets/s', 'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count', 'URG_Flag_Count', 'ECE_Flag_Count', 'Down/Up_Ratio', 'Avg_Fwd_Segment_Size', 'Fwd_Header_Length.1', 'Subflow_Fwd_Packets', 'Subflow_Fwd_Bytes', 'Subflow_Bwd_Packets', 'Subflow_Bwd_Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active_Mean', 'Active_Std', 'Idle_Std']
Número de características eliminadas:
38


Let's now re-draw the updated correlation matrix:

In [ ]:
df_ids_filtered_corr = df_ids_filtered.corr()
fig = px.imshow(df_ids_filtered_corr, color_continuous_scale='Viridis', labels=dict(color='Correlación'))
fig.update_layout(title='Matriz de Correlación', width=800, height=800)
fig.show()

print('Número de características:')
print(df_ids_filtered.shape[1])

Número de características:
15


Additionally, based on the correlation matrix and the definition of the features, we will manually remove the following features, which are already represented as a combination of others:

* Packet_Length_Variance
* Average_Packet_Size

Regarding correlations, we observe that the following feature has a correlation value with `'Label'` slightly above 0.2, so we proceed to remove it:

* Flow_IAT_Mean


In [ ]:
caracteristicas_a_eliminar_2 = ['Packet_Length_Variance','Average_Packet_Size','Flow_IAT_Mean']
# Eliminar las características seleccionadas del DataFrame
df_ids_filtered = df_ids_filtered.drop(columns=caracteristicas_a_eliminar_2)

To check if any more features can be eliminated, we will calculate the variance values of the remaining features and compare them.

Features with a very low relative variance (compared to the rest) will be candidates for removal. Initially, we display a list of features and their variances:


In [ ]:
# Calcular la varianza de cada característica en df_ids_filtered
df_ids_var = df_ids_filtered.drop(columns='Label')
variances = df_ids_var.var()

# Imprimir los valores de varianza de todas las características
print("Valores de Varianza de Todas las Características:")
print(variances)

Valores de Varianza de Todas las Características:
Flow_Duration             1.188169e+15
Bwd_Packet_Length_Mean    3.820449e+05
Bwd_Packet_Length_Std     7.352249e+05
Flow_IAT_Std              6.823012e+13
Fwd_IAT_Total             1.183111e+15
Fwd_IAT_Std               9.816473e+13
Packet_Length_Mean        9.687686e+04
Packet_Length_Std         4.147013e+05
FIN_Flag_Count            3.000341e-02
Avg_Bwd_Segment_Size      3.820449e+05
Idle_Mean                 5.909057e+14
dtype: float64


The feature `'FIN_Flag_Count'` shows a variance several orders of magnitude lower than the rest. Based on the variance selection criterion, we can eliminate this feature—an argument reinforced by its relatively low correlation with `'Label'` (0.25).



In [ ]:
caracteristicas_a_eliminar_3 = ['FIN_Flag_Count']
# Eliminar las características seleccionadas del DataFrame
df_ids_filtered = df_ids_filtered.drop(columns=caracteristicas_a_eliminar_3)

The resulting correlation matrix is as follows:


In [ ]:
df_ids_filtered_corr = df_ids_filtered.corr()
fig = px.imshow(df_ids_filtered_corr, color_continuous_scale='Viridis', labels=dict(color='Correlación'))
fig.update_layout(title='Matriz de Correlación', width=800, height=800)
fig.show()

print('Número de características:')
print(df_ids_filtered.shape[1])

Número de características:
11


We now list the surviving features from the screening to select the columns we need from the general dataframe:


In [ ]:
# Listar las columnas del DataFrame
columnas_df_ids_filtered = df_ids_filtered.columns

# Imprimir las columnas
print("Columnas del DataFrame:")
for columna in columnas_df_ids_filtered:
    print(columna)

Columnas del DataFrame:
Flow_Duration
Bwd_Packet_Length_Mean
Bwd_Packet_Length_Std
Flow_IAT_Std
Fwd_IAT_Total
Fwd_IAT_Std
Packet_Length_Mean
Packet_Length_Std
Avg_Bwd_Segment_Size
Idle_Mean
Label


From the complete dataframe `df_ids`, we keep only the columns resulting from the feature study and display the first ten records

In [ ]:
# Seleccionamos solo las columnas especificadas utilizando la indexación directa:
df_ids = df_ids[columnas_df_ids_filtered]
df_ids.head(10)

,Flow_Duration,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_IAT_Std,Fwd_IAT_Total,Fwd_IAT_Std,Packet_Length_Mean,Packet_Length_Std,Avg_Bwd_Segment_Size,Idle_Mean,Label
1090143,99632,0.0,0.000000,2.076839e+04,99632,2.076839e+04,24.000000,0.000000,0.0,0.0,0
67513,186,112.0,0.000000,6.710440e+01,3,0.000000e+00,76.000000,32.863353,112.0,0.0,0
2113229,98463165,1932.5,2848.073577,2.970000e+07,98500000,4.400000e+07,919.846154,2083.276620,1932.5,98400000.0,1
2599722,627,2.0,0.000000,2.566798e+02,627,0.000000e+00,3.000000,2.000000,2.0,0.0,1
1822668,82222,140.0,0.000000,0.000000e+00,0,0.000000e+00,84.666667,47.920072,140.0,0.0,0
179581,3543632,46.0,0.000000,1.452867e+06,3543632,1.615591e+06,32.428571,18.866700,46.0,0.0,0
661395,243,79.0,0.000000,5.629387e+01,48,0.000000e+00,52.000000,24.647515,79.0,0.0,0
533657,49,0.0,0.000000,0.000000e+00,49,0.000000e+00,2.000000,0.000000,0.0,0.0,0
2502610,50251,79.0,0.000000,2.896740e+04,4,0.000000e+00,62.200000,15.336232,79.0,0.0,0
244343,3429510,0.0,0.000000,1.713724e+06,3429510,1.713724e+06,6.000000,0.000000,0.0,0.0,1



---

# Outlier Analysis

The outlier analysis will be conducted using the Interquartile Range (IQR) method, where we calculate the interquartile range for each numerical column and manually set limits for detecting records that fall far from the rest.

In our case, we will calculate the 1st percentile to get an initial idea of the outliers' appearance.
We start by calculating the interquartile range for each numerical column:


In [ ]:
Q1 = df_ids.quantile(0.01)
Q3 = df_ids.quantile(0.99)
IQR = Q3 - Q1

We define limits to identify outliers using the IQR criterion.




In [ ]:
lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR


We identify the rows that contain the outliers detected in this study:


In [ ]:
outliers = ((df_ids < lower_bound) | (df_ids > upper_bound)).any(axis=1)
# Contar el número de registros con outliers (True en la máscara 'outliers')
conteo_outliers = outliers.sum()
print(f"Total de registros con outliers (True): {conteo_outliers}")

Total de registros con outliers (True): 374


We create a dataset that contains only the outliers:


In [ ]:
# Mostrar las filas que contienen outliers True
filas_con_outliers = df_ids[outliers]
filas_con_outliers

,Flow_Duration,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_IAT_Std,Fwd_IAT_Total,Fwd_IAT_Std,Packet_Length_Mean,Packet_Length_Std,Avg_Bwd_Segment_Size,Idle_Mean,Label
1020085,7010,0.0,0.0,2.082626e+03,7010,2232.423272,3337.142857,3233.417770,0.0,0.0,0
2421157,104766552,6.0,0.0,7.410000e+07,105000000,0.000000,117.250000,230.517353,6.0,105000000.0,1
2507905,107647795,6.0,0.0,7.610000e+07,108000000,0.000000,112.000000,220.018181,6.0,108000000.0,1
2415297,111771692,6.0,0.0,7.900000e+07,112000000,0.000000,178.250000,352.511347,6.0,112000000.0,1
1920182,115872142,0.0,0.0,8.190000e+07,116000000,0.000000,4.000000,4.618802,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
2512380,100914799,6.0,0.0,7.140000e+07,101000000,0.000000,88.000000,172.023254,6.0,101000000.0,1
2414072,111752471,6.0,0.0,7.900000e+07,112000000,0.000000,110.500000,217.018432,6.0,112000000.0,1
1013260,9977,0.0,0.0,6.572311e+02,9977,664.774690,2711.428571,2930.410015,0.0,0.0,0
2507090,107485180,6.0,0.0,7.600000e+07,107000000,0.000000,112.250000,220.518140,6.0,107000000.0,1


**Conclusion:**
As can be observed and in line with technical intuition, filtering outliers is mainly the identification of the attacks themselves, which can be seen in an inspection of the resulting outliers dataframe `filas_con_outliers`.

Therefore, we decide not to eliminate outliers for two reasons:
- Records with attacks must be preserved for better model training, and an outlier filter would remove these records.
- Benign outliers are a minority: the presence of these is considered less important due to the robustness and precision of the models (Random Forest, kNN, CNN) in handling outliers.


---

# Data Export

---


We divide the dataset `df_ids` into four datasets of similar size:



In [ ]:
# Obtener la cantidad total de filas en el DataFrame
total_filas = len(df_ids)

# Calcular el tamaño aproximado de cada división
div_size = total_filas // 4  # Dividir en cuatro partes

# Dividir el DataFrame en cuatro datasets con tamaño similar
datasets_divididos = np.array_split(df_ids, 4)

# Asignar los nombres a los datasets resultantes
df_ids_0, df_ids_1, df_ids_2, df_ids_3 = datasets_divididos

We print the number of rows and columns of the resulting dataframes:

In [ ]:
print("Shape of df_ids_0", df_ids_0.shape)
print("Shape of df_ids_1", df_ids_1.shape)
print("Shape of df_ids_2", df_ids_2.shape)
print("Shape of df_ids_3", df_ids_3.shape)

Shape of df_ids_0 (630503, 11)
Shape of df_ids_1 (630502, 11)
Shape of df_ids_2 (630502, 11)
Shape of df_ids_3 (630502, 11)


Next, we perform a check of the attack count for each of the resulting dataframes:


In [ ]:
# Definir una función para contar valores 'Label' igual a 1 en un DataFrame
def contar_valores_label_1(df):
    return (df['Label'] == 1).sum()

# Calcular y mostrar el número de valores 'Label' igual a 1 para cada DataFrame
print("Número de valores 'Label' igual a 1 en df_ids_0:", contar_valores_label_1(df_ids_0))
print("Número de valores 'Label' igual a 1 en df_ids_1:", contar_valores_label_1(df_ids_1))
print("Número de valores 'Label' igual a 1 en df_ids_2:", contar_valores_label_1(df_ids_2))
print("Número de valores 'Label' igual a 1 en df_ids_3:", contar_valores_label_1(df_ids_3))

Número de valores 'Label' igual a 1 en df_ids_0: 111670
Número de valores 'Label' igual a 1 en df_ids_1: 107160
Número de valores 'Label' igual a 1 en df_ids_2: 104414
Número de valores 'Label' igual a 1 en df_ids_3: 102631


We export the files to `.csv` format:

*Note: Each file specifies its number (from 0 to 3) and whether the dataframe is balanced or not (NB: Not balanced, B: Balanced).*


In [ ]:
df_ids_0.to_csv('df_ids_0_NB_v1.csv', index=False)
df_ids_1.to_csv('df_ids_1_NB_v1.csv', index=False)
df_ids_2.to_csv('df_ids_2_NB_v1.csv', index=False)
df_ids_3.to_csv('df_ids_3_NB_v1.csv', index=False)


**Data Balancing Before Export:**

Below are the same four dataframes with balanced data (approximately the same number of benign and attack records).

For balancing, the technique used is *downsampling*. Due to the abundance of data, it is estimated that the removal of records from the benign class (0) will not significantly impact the detection of this class by the models.


In [ ]:
# Definir una función para balancear un DataFrame según la columna 'Label'
def balancear_dataframe(df):
    # Contar el número de registros para cada valor de 'Label'
    conteo_clase_0 = (df['Label'] == 0).sum()
    conteo_clase_1 = (df['Label'] == 1).sum()

    # Determinar el número mínimo de registros a mantener para igualar las clases
    min_registros = min(conteo_clase_0, conteo_clase_1)

    # Filtrar y seleccionar aleatoriamente 'min_registros' registros para cada clase
    df_clase_0 = df[df['Label'] == 0].sample(n=min_registros, random_state=3)
    df_clase_1 = df[df['Label'] == 1].sample(n=min_registros, random_state=3)

    # Concatenar los DataFrames balanceados
    df_balanceado = pd.concat([df_clase_0, df_clase_1])

    return df_balanceado

# Balancear cada DataFrame df_ids_0, df_ids_1, df_ids_2, df_ids_3
df_ids_0_balanceado = balancear_dataframe(df_ids_0)
df_ids_1_balanceado = balancear_dataframe(df_ids_1)
df_ids_2_balanceado = balancear_dataframe(df_ids_2)
df_ids_3_balanceado = balancear_dataframe(df_ids_3)

# Mostrar información sobre los DataFrames balanceados
print("Información de df_ids_0_balanceado:")
print(df_ids_0_balanceado['Label'].value_counts())
print()

print("Información de df_ids_1_balanceado:")
print(df_ids_1_balanceado['Label'].value_counts())
print()

print("Información de df_ids_2_balanceado:")
print(df_ids_2_balanceado['Label'].value_counts())
print()

print("Información de df_ids_3_balanceado:")
print(df_ids_3_balanceado['Label'].value_counts())

Información de df_ids_0_balanceado:
Label
0    111670
1    111670
Name: count, dtype: int64

Información de df_ids_1_balanceado:
Label
0    107160
1    107160
Name: count, dtype: int64

Información de df_ids_2_balanceado:
Label
0    104414
1    104414
Name: count, dtype: int64

Información de df_ids_3_balanceado:
Label
0    102631
1    102631
Name: count, dtype: int64


We export these four balanced dataframes to `.csv` files:


In [ ]:
df_ids_0_balanceado.to_csv('df_ids_0_B_v1.csv', index=False)
df_ids_1_balanceado.to_csv('df_ids_1_B_v1.csv', index=False)
df_ids_2_balanceado.to_csv('df_ids_2_B_v1.csv', index=False)
df_ids_3_balanceado.to_csv('df_ids_3_B_v1.csv', index=False)

We provide, as a code comment, the `!gdown` statement to download a .zip file containing all the .csv files resulting from the data processing:


In [ ]:
# Enlace de Google Drive:
# https://drive.google.com/file/d/1Ki4Qcb-xaNlCHWypLpzH-aNHFWPLVRm4/view?usp=sharing

# Sentencia Gdown:
# !gdown 1Ki4Qcb-xaNlCHWypLpzH-aNHFWPLVRm4

We remove data from the previous code (except for the output) to free up RAM:

In [ ]:
del df_ids